# Rapport du Défi de Régression

**Membres du groupe :**
- Adam Schlee
- Truong Son Ngo
- Jixiang Sun
- Thi Tho Vu
- Mohamed Fakroni
- Mateo Carvajal Sanchez
- Huu Thanh Tu Huynh
- Santiago Forero Gutierrez

Ce notebook présente l'approche utilisée pour relever le défi de régression. Il inclut le chargement et le prétraitement des données, la définition d’un pipeline et d’une grille d’hyperparamètres, le tuning avec HalvingRandomSearchCV, l’entraînement du modèle final, ainsi qu’un système interactif de sélection de modèles pour comparer différentes approches.

## I. Introduction

L'objectif de ce projet est de prédire la variable `co2` à partir d'un ensemble de variables explicatives. Plusieurs modèles de régression ont été testés et comparés (ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, VotingRegressor, StackingRegressor, XGBRegressor, LightGBM). Le présent document détaille les étapes du prétraitement, la recherche d’hyperparamètres, le tuning du modèle et la génération du fichier de soumission.

## II. Import des Bibliothèques et Chargement des Données

Les bibliothèques nécessaires sont importées et les données de régression sont chargées depuis les fichiers CSV.

In [ ]:
#! pip install numpy
#! pip install pandas
#! pip install scikit-learn
#! pip install xgboost
#! pip install lightgbm

import numpy as np
import pandas as pd
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, RandomizedSearchCV, HalvingRandomSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.utils import resample
import time

##### Chargement des données
train_df = pd.read_csv("../data/regression/train.csv")
test_df = pd.read_csv("../data/regression/test.csv")
sample_submission = pd.read_csv("../data/regression/sample_submission.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

## III. Prétraitement des Données

Les données d'entraînement et de test sont concaténées pour appliquer un prétraitement commun :
- Ajout d'un indicateur pour différencier les ensembles
- Traitement spécifique de la variable `hc`
- Imputation des valeurs manquantes pour les variables numériques et catégoriques
- Encodage one-hot pour les variables catégoriques
- Séparation finale en ensembles d'entraînement et de test

In [ ]:
##### Ajout d'un indicateur pour différencier train/test et traitement de 'co2'
train_df["is_train"] = 1
test_df["is_train"] = 0
test_df["co2"] = np.nan
data = pd.concat([train_df, test_df], sort=False)
data_index = data["id"]
data = data.drop("id", axis=1)

##### Traitement de la variable 'hc'
data['hc'] = data['hc'].fillna(data['hcnox'] - data['nox'])
data = data.drop(columns=['hcnox'])

##### Imputation pour les variables numériques et catégoriques
numeric_cols = data.select_dtypes(include=["float64", "int64"]).columns.tolist()
categorical_cols = data.select_dtypes(exclude=["float64", "int64"]).columns.tolist()

for col in numeric_cols:
    data[col] = data[col].fillna(data[col].median())
for col in categorical_cols:
    data[col] = data[col].fillna("missing")

##### Encodage one-hot pour les variables catégoriques
data = pd.get_dummies(data, drop_first=True)

##### Séparation train/test
train_data = data[data["is_train"] == 1].drop("is_train", axis=1)
test_data = data[data["is_train"] == 0].drop(["is_train", "co2"], axis=1)
y_train = train_data["co2"]
X_train = train_data.drop("co2", axis=1)

print("Processed X_train shape:", X_train.shape)
print("Processed test_data shape:", test_data.shape)

## IV. Recherche d'Hyperparamètres et Tuning

Pour accélérer le tuning, nous sélectionnons d'abord un sous-échantillon (15 000 observations) de l’ensemble d'entraînement. Un pipeline simple, composé d'un `StandardScaler` et d'un `ExtraTreesRegressor`, est défini. Ensuite, une grille d'hyperparamètres est construite pour `ExtraTreesRegressor` et optimisée via `HalvingRandomSearchCV`.

In [ ]:
##### Sous-échantillonnage pour le tuning
X_train_sample, y_train_sample = resample(X_train, y_train, n_samples=15000, random_state=42)

##### Définition du pipeline
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ExtraTreesRegressor(random_state=42, n_jobs=-1))
])

##### Définition de la grille d'hyperparamètres
param_grid = {
    "model__n_estimators": [7, 8, 9, 10, 11],
    "model__max_depth": [None, 5, 10, 15, 20, 25, 30],
    "model__min_samples_split": [2, 3, 4, 5],
    "model__min_samples_leaf": [1, 2, 3, 4, 5],
    "model__max_features": ["sqrt", "log2", None, 0.5, 0.75, 0.9],
    "model__bootstrap": [True, False],
    "model__criterion": ["squared_error"]
}

##### Optimisation avec HalvingRandomSearchCV
tuner = HalvingRandomSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    factor=3,                              
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
    cv=3,
    verbose=1,
    random_state=42,
    max_resources="auto"
)

##### Mesure du temps de tuning
start_time = time.time()
tuner.fit(X_train_sample, y_train_sample)
tuning_time = time.time() - start_time
print(f"Temps de tuning sur l'échantillon: {tuning_time:.2f} secondes")

##### Affichage des meilleurs hyperparamètres
print("Meilleurs paramètres trouvés :", tuner.best_params_)

##### Extraction des paramètres (suppression du préfixe 'model__')
best_model_params = {key.replace("model__", ""): value 
                     for key, value in tuner.best_params_.items() 
                     if key.startswith("model__")}
print("Paramètres pour ExtraTreesRegressor :", best_model_params)

## V. Entraînement du Modèle Final et Génération du Fichier de Soumission

Une fois les hyperparamètres optimaux obtenus pour `ExtraTreesRegressor`, le modèle final est entraîné sur l’ensemble complet d'entraînement. Par ailleurs, un système interactif permet de sélectionner d'autres modèles (RandomForest, GradientBoosting, AdaBoost, etc.) pour des tests rapides avant de générer le fichier de soumission.

In [ ]:
##### Entraînement du modèle final avec ExtraTreesRegressor
final_model = ExtraTreesRegressor(n_estimators=9, random_state=17, n_jobs=-1)   # Les meilleurs hyperparamètres trouvés
final_model.fit(X_train, y_train)

##### Prédictions sur l'ensemble de test
test_preds = final_model.predict(test_data)

##### Création du fichier de soumission (les prédictions sont converties en entiers)
submission = pd.DataFrame({
    "id": test_df["id"],
    "co2": test_preds.astype(int)
})
submission.to_csv("../result/regression/Final_ensemble_submission.csv", index=False)
print("Submission saved as Final_ensemble_submission.csv")

##### Système interactif de sélection de modèle
print("Which model would you like to use?")
print("1. RandomForestRegressor")
print("2. GradientBoostingRegressor")
print("3. AdaBoostRegressor")
print("4. ExtraTreesRegressor")
print("5. BaggingRegressor")
print("6. VotingRegressor")
print("7. StackingRegressor")
print("8. XGBRegressor")

model = 0

while model == 0:
    model_choice = input(": ")
    
    match model_choice:
        case "1":
            from sklearn.ensemble import RandomForestRegressor
            model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        case "2":
            from sklearn.ensemble import GradientBoostingRegressor
            model = GradientBoostingRegressor(n_estimators=100, random_state=42)
        case "3":
            from sklearn.ensemble import AdaBoostRegressor
            model = AdaBoostRegressor(n_estimators=100, random_state=42)
        case "4":
            model = ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        case "5":
            from sklearn.ensemble import BaggingRegressor
            model = BaggingRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        case "6":
            from sklearn.ensemble import VotingRegressor
            model = VotingRegressor(estimators=[
                ("rf", RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
                ("gb", GradientBoostingRegressor(n_estimators=100, random_state=42))
            ], n_jobs=-1)
        case "7":
            from sklearn.ensemble import StackingRegressor
            model = StackingRegressor(estimators=[
                ("rf", RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
                ("gb", GradientBoostingRegressor(n_estimators=100, random_state=42))
            ], final_estimator=RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1), n_jobs=-1)
        case "8":
            model = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        case _:
            print("Invalid choice. Please select a valid model.")
            model = 0

print("Model chosen:", model)

##### Test rapide sur un sous-échantillon pour vérifier le modèle choisi
X_train_sample, y_train_sample = resample(X_train, y_train, n_samples=5000, random_state=42)
model.fit(X_train_sample, y_train_sample)
y_pred_sample = model.predict(X_train_sample)
mae_sample = mean_absolute_error(y_train_sample, y_pred_sample)
print("Quick test MAE on sample data: {:.4f}".format(mae_sample))

##### Entraînement final du modèle sélectionné et prédictions sur le test
model.fit(X_train, y_train)
est_preds = model.predict(test_data)

##### Création du fichier de soumission avec le nom du modèle utilisé
model_name = str(model).split('(')[0]
submission = pd.DataFrame({
    "id": sample_submission["id"],
    "co2": est_preds.astype(int)
})
submission.to_csv(f"../result/regression/{model_name}_submission.csv", index=False)
print(f"Submission saved as {model_name}_submission.csv")

## VI. Conclusion

Ce défi de régression nous a permis d’explorer plusieurs techniques de modélisation. L’optimisation des hyperparamètres via HalvingRandomSearchCV a aidé à sélectionner rapidement une configuration efficace pour ExtraTreesRegressor. De plus, le système interactif permet de comparer différentes approches afin de choisir le modèle le plus adapté aux caractéristiques des données. Le fichier de soumission final est généré sous le format requis pour la compétition.